In [7]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

In [8]:
# Load dataset (replace 'house_data.csv' with your actual file path)
df = pd.read_csv('Housing.csv')

# Inspect the data
print(df.head())
print(df.info())
print(df.describe())

      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 colu

In [9]:
# Handle missing values (fill numerical features with median, categorical features with mode)
for col in df.columns:
    if df[col].dtype == 'object':  # Check if column is categorical
        df[col].fillna(df[col].mode()[0], inplace=True) # Fill with mode
    else:
        df[col].fillna(df[col].median(), inplace=True) # Fill with median

In [10]:
# One-hot encode categorical variables
df = pd.get_dummies(df, drop_first=True)

In [11]:
# Log transform the target variable (if skewed)
df['log_price'] = np.log(df['price'])
df.drop('price', axis=1, inplace=True)

In [12]:
# Feature and target split
X = df.drop('log_price', axis=1)
y = df['log_price']

In [13]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Standardize the numerical features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
# Model 1: Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [16]:
# Predictions
rf_predictions = rf_model.predict(X_test)

In [17]:
# Model Evaluation (Random Forest)
rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
rf_r2 = r2_score(y_test, rf_predictions)
print(f"Random Forest MAE: {rf_mae}")
print(f"Random Forest RMSE: {rf_rmse}")
print(f"Random Forest R²: {rf_r2}")

Random Forest MAE: 0.20874143161387806
Random Forest RMSE: 0.266195303341503
Random Forest R²: 0.6330187759360999


In [18]:
# Model 2: XGBoost Regressor
xgb_model = xgb.XGBRegressor(n_estimators=500, learning_rate=0.05, random_state=42)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [19]:
# Predictions
xgb_predictions = xgb_model.predict(X_test)

In [20]:
# Model Evaluation (XGBoost)
xgb_mae = mean_absolute_error(y_test, xgb_predictions)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_predictions))
xgb_r2 = r2_score(y_test, xgb_predictions)
print(f"XGBoost MAE: {xgb_mae}")
print(f"XGBoost RMSE: {xgb_rmse}")
print(f"XGBoost R²: {xgb_r2}")

XGBoost MAE: 0.21507484845568228
XGBoost RMSE: 0.2746923916101832
XGBoost R²: 0.6092164006215767


In [23]:
# Hyperparameter tuning using GridSearchCV for Random Forest
param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
best_rf_model = grid_search.best_estimator_

In [25]:
# Predictions with the best model
best_rf_predictions = best_rf_model.predict
print(best_rf_predictions)

<bound method ForestRegressor.predict of RandomForestRegressor(max_depth=10, min_samples_split=10, n_estimators=500,
                      random_state=42)>
